In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'): 
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-eight/Data.xlsx
/kaggle/input/testingdata2/UABComments.csv
/kaggle/input/dataseven/Data.xlsx


In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/data-eight/Data.xlsx
/kaggle/input/testingdata2/UABComments.csv
/kaggle/input/dataseven/Data.xlsx


In [3]:
import pandas as pd
df = pd.read_excel('/kaggle/input/data-eight/Data.xlsx')
df.head()


,Reviews,Category
0,Arena club food buffet has really stepped up a...,Concessions
1,Arena Club staff catering staff very helpful,Concessions
2,Have never used.,Concessions
3,Stm meals have been great this year. Please br...,Concessions
4,We do not buy from the concession.,Concessions


In [4]:
df['Category'].value_counts()

Category
Concessions          270
Video Board/Sound     86
Cheer                 85
Ticketing             79
Fac.                  75
Marketing             73
Parking               68
Name: count, dtype: int64

In [5]:
print(df.columns)

Index(['Reviews', 'Category'], dtype='object')


In [6]:
# Split into features (reviews) and labels (categories)
X = df['Reviews']
Y = df['Category']
len(X)

736

In [7]:
# Split the data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=22)

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report 

In [9]:

# Define pipelines for different models
pipeMNB = Pipeline([('tfidf', TfidfVectorizer(stop_words='english', ngram_range=(1, 2))), ('clf', MultinomialNB())])
pipeCNB = Pipeline([('tfidf', TfidfVectorizer(stop_words='english', ngram_range=(1,3))), ('clf', ComplementNB())])
pipeSVC = Pipeline([('tfidf', TfidfVectorizer(stop_words='english', ngram_range=(1,3))), ('clf', LinearSVC())])

In [10]:
print(df.isnull().sum())

Reviews     0
Category    0
dtype: int64


In [11]:
df['Reviews'] = df['Reviews'].fillna('')

In [12]:
df['Reviews'] = df['Reviews'].astype(str)

In [13]:
print(X_train.isnull().sum())  # Check for remaining missing values
print(X_train.dtypes)          # Check data types

0
object


In [14]:
pipeMNB.fit(X_train, Y_train)
predictMNB = pipeMNB.predict(X_test)
print(f"MNB Accuracy: {accuracy_score(Y_test, predictMNB):.2f}")

pipeCNB.fit(X_train, Y_train)
predictCNB = pipeCNB.predict(X_test)
print(f"CNB Accuracy: {accuracy_score(Y_test, predictCNB):.2f}")

pipeSVC.fit(X_train, Y_train)
predictSVC = pipeSVC.predict(X_test)
print(f"SVC Accuracy: {accuracy_score(Y_test, predictSVC):.2f}")

MNB Accuracy: 0.45
CNB Accuracy: 0.80
SVC Accuracy: 0.82


In [15]:
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

# Encode the target labels
label_encoder = LabelEncoder()
Y_train_encoded = label_encoder.fit_transform(Y_train)
Y_test_encoded = label_encoder.transform(Y_test)

# XGBoost pipeline
pipeXGB = Pipeline([('tfidf', TfidfVectorizer(stop_words = 'english')),
                    ('clf', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))])

# Train the model
pipeXGB.fit(X_train, Y_train_encoded)  
predictXGB = pipeXGB.predict(X_test)

# Decode the predictions back to original labels
predictXGB_decoded = label_encoder.inverse_transform(predictXGB)

# Print accuracy
print(f"XGB Accuracy: {accuracy_score(Y_test, predictXGB_decoded):.2f}") 

XGB Accuracy: 0.72


In [16]:
msg = "There was a big queue in ticketing office and tickets were expensive"
pipeSVC.predict([msg]) 

array(['Ticketing'], dtype=object)

In [17]:
msg = "It would be more entertaining to hear the band or the DJ during timeouts. There is too much talking and too many promotions. It kills the vibe."
pipeCNB.predict([msg]) 

array(['Cheer'], dtype='<U17')

In [18]:
msg = "I know this is something outside control, but the TV timeouts while on a subscription service are getting way out of hand.  Constant commercial breaks make game feel dead and drag on forever. No reason a game on ESPN+ should take almost 4 1/2 hours.  "
pipeSVC.predict([msg]) 

array(['Fac.'], dtype=object)

In [19]:
msg = "The stadium needs affordable food options. The prices are too high. "
pipeSVC.predict([msg]) 

array(['Concessions'], dtype=object)

In [20]:
msg = "The seats were quite hard to sit on, and the restroom wasnt clean"
pipeCNB.predict([msg]) 

array(['Fac.'], dtype='<U17')

In [21]:
import pandas as pd

# Load the new data with comments to classify
new_data = pd.read_csv('/kaggle/input/testingdata2/UABComments.csv', encoding='ISO-8859-1')

new_data['Comments'] = new_data['Comments'].fillna('')  # Fill any missing values

# Check the data
print(new_data.head())

# Predict using the best-performing model (pipeSVC in this case)
predictions = pipeSVC.predict(new_data['Comments']) 

# Add the predictions as a new column in the DataFrame 
new_data['Predicted_Category'] = predictions
print(new_data[['Comments', 'Predicted_Category']].head()) 
# Save the classified data to a new CSV file
new_data.to_csv('/kaggle/working/classified_comments.csv', index=False)

print("Predictions saved to classified_comments.csv")

                                            Comments
0  Nothing open on the concessions.  Limited opti...
1                               Better food choices.
2    $14.00 FOR ONE REFILLABLE DRINK IS OUTRAGEOUS. 
3  prices of concession are obscene. People servi...
4  1.  There are so many video boards in the stad...
                                            Comments Predicted_Category
0  Nothing open on the concessions.  Limited opti...        Concessions
1                               Better food choices.        Concessions
2    $14.00 FOR ONE REFILLABLE DRINK IS OUTRAGEOUS.         Concessions
3  prices of concession are obscene. People servi...        Concessions
4  1.  There are so many video boards in the stad...  Video Board/Sound
Predictions saved to classified_comments.csv


In [22]:
import os

# List the files in the current working directory
print(os.listdir('/kaggle/working'))

['__notebook__.ipynb', 'classified_comments.csv']


In [23]:
# Importing necessary libraries
import pandas as pd
import spacy

# Load SpaCy's English language model
nlp = spacy.load("en_core_web_sm")

# Load your dataset
# Assuming the dataset is named 'reviews.csv' with two columns: 'reviews' and 'categories'
df = pd.read_excel("/kaggle/input/data-eight/Data.xlsx")

# Function to extract keywords using SpaCy
def extract_keywords(text):
    doc = nlp(text)
    # Extracting nouns, proper nouns, and adjectives as keywords
    keywords = [token.text for token in doc if token.pos_ in ('NOUN', 'PROPN', 'ADJ')]
    # Return summarized words as a single string
    return ', '.join(keywords)

# Apply the function to the reviews column and create a new column for keywords
df['keywords'] = df['Reviews'].apply(extract_keywords)

# Save the updated DataFrame to a new CSV file
output_path = "/kaggle/working/reviews_with_keywords.csv"
df.to_csv(output_path, index=False)

print(f"Keyword extraction complete. File saved to {output_path}") 

Keyword extraction complete. File saved to /kaggle/working/reviews_with_keywords.csv
